In [1]:
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

ISBN = 9789869109321  # 二十一世紀資本論

df_lst = []  # 裝著各種表格的 list

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
driver = webdriver.Chrome(options=my_options)

In [2]:
# 世新大學
try:
    url = "https://lib.shu.edu.tw/"
    driver.get(url)

    # 定位選擇欄，以 ISBN 搜尋方式搜尋
    select = Select(driver.find_element_by_name("idx"))
    select.select_by_visible_text(u"ISBN")
    search_input = driver.find_element_by_name("q")
    search_input.send_keys(ISBN)
    search_input.submit()

    # 遇到「你要提交的資訊未受到保護」，點選仍要傳送
    continue_btn = driver.find_element_by_xpath('//*[@id=\"proceed-button\"]')
    continue_btn.click()

    url = driver.current_url
    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    df_shu = dfs[0]

    df_shu.insert(0, "圖書館", ["世新大學" for i in range(df_shu.shape[0])])
    for deleted in ["館藏", "到期日", "條碼", "館藏預約"]:
        df_shu.pop(deleted)
    df_shu.rename(columns={"現行位置": "館藏地",
                           "狀態": "館藏狀態"}, inplace=True)
    df_shu["館藏地"] = df_shu["館藏地"].str.replace("世新大學圖書館", "")
    df_lst.append(df_shu)
except:
    print(f"「{url}」無法爬取！")

In [3]:
# 國立臺北大學
try:
    url = f"http://webpac.lib.ntpu.edu.tw/search.cfm?m=ss&k0={ISBN}&t0=k&c0=and"
    driver.get(url)

    html_text = driver.page_source
    soup = BeautifulSoup(html_text, "html.parser")

    # 會搜尋到多本書，需要取得每本書的網址
    hrefs = []
    list_boxes = soup.find_all("div", "list_box")
    for list_box in list_boxes:
        hrefs.append("http://webpac.lib.ntpu.edu.tw/" + list_box.find("a", "btn")["href"])

    # 開始爬表格
    df_ntpu = pd.DataFrame()
    for url in hrefs:
        # 進入頁面爬取表格
        dfs = pd.read_html(url, encoding="utf-8")
        df_temp = dfs[0]

        # 新增欄位《館藏地》
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "html.parser")
        place_dad = soup.find("div", "book_area").find("h2").text
        place_son = soup.find("div", "book_area").find("h3").text
        df_temp.insert(1, "館藏地", [place_dad + place_son for i in range(df_temp.shape[0])])

        # 合併表格
        df_ntpu = pd.concat([df_ntpu, df_temp], axis=0, ignore_index=True)

    # 新增欄位《圖書館》
    df_ntpu.insert(0, "圖書館", ["國立臺北大學" for i in range(df_ntpu.shape[0])])
    # 刪除不要的欄位
    for deleted in ["登錄號", "資料類型", "使用類型", "附件", "預約人數", "Unnamed: 7"]:
        df_ntpu.pop(deleted)
    df_lst.append(df_ntpu)
except:
    print(f"「{url}」無法爬取！")

In [4]:
# 國立臺灣博物館
try:
    # ISBN = 9789865321703
    url = "https://lib.moc.gov.tw/F"
    driver.get(url)

    time.sleep(1)  # 為了等待網頁加載

    select = Select(driver.find_element_by_name("x"))
    select.select_by_visible_text(u"ISBN")
    search_input = driver.find_element_by_name("y")
    search_input.send_keys(ISBN)
    # search_input.submit()  # 不知道為什麼無法 submit()？
    submit_input = driver.find_element_by_name("Search")
    submit_input.click()

    click = driver.find_element_by_xpath("/html/body/table[9]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/a")
    click.click()

    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    df_ntm = dfs[11]

    # 清理資料
    df_ntm = df_ntm.drop(df_ntm.columns[[0, 1, 5, 6, 8, 9, 10]], axis=1)
    # 處理有 JS 語法問題
    for i in range(df_ntm.shape[0]):
        df_ntm.at[i, "到期日"] = df_ntm.at[i, "到期日"].split(";")[-1].strip()

    df_ntm.rename(columns={"館藏地": "圖書館",
                            "典藏區域": "館藏地",
                            "到期日": "館藏狀態"}, inplace=True)
    df_ntm["圖書館"] = df_ntm["圖書館"].str.replace("臺史博", "國立臺灣博物館")
    df_lst.append(df_ntm)
except:
    print(f"「{url}」無法爬取！")

「https://lib.moc.gov.tw/F」無法爬取！


In [5]:
df_final = pd.concat(df_lst, axis=0, ignore_index=True)
df_final

,圖書館,館藏地,索書號,館藏狀態
0,世新大學,中文圖書區,550.187 8757 (瀏覽書架),可借
1,國立臺北大學,三峽圖書館3F中文圖書區,550.187 4025 107,仍在館內 Available
2,國立臺北大學,三峽圖書館3F中文圖書區,550.187 4025,仍在館內 Available
3,國立臺北大學,三峽圖書館3F中文圖書區,550.187 4025 c.2,仍在館內 Available
4,國立臺北大學,三峽圖書館3F中文圖書區,550.187 4025 c.3,已被外借 On loan 到期日 2021-07-26


In [6]:
df_final.to_csv("df_final.csv", encoding="big5", index=True)